In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def gdp_money_sim(money_growth=0.05, inflation=0.03, T=10):
    T = int(T)  # ← FIX HERE
    years = list(range(2020, 2020 + T))
    nominal = [1000 * (1 + money_growth)**t for t in range(T)]
    real = [nominal[t] / (1 + inflation)**t for t in range(T)]

    plt.figure(figsize=(10,5))
    plt.plot(years, nominal, label='Nominal GDP (Cash)', linewidth=2, color='gold')
    plt.plot(years, real, label='Real GDP (Stuff)', linewidth=2, color='seagreen')
    plt.fill_between(years, real, nominal, color='lightgray', alpha=0.4, label='Inflation Zone')
    
    plt.title("💵 Nominal vs. Real GDP Over Time")
    plt.xlabel("Year")
    plt.ylabel("GDP ($)")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.show()

interact(gdp_money_sim,
         money_growth=FloatSlider(value=0.05, min=0, max=0.15, step=0.01, description='Money Growth'),
         inflation=FloatSlider(value=0.03, min=0, max=0.15, step=0.01, description='Inflation'),
         T=FloatSlider(value=10, min=5, max=30, step=1, description='Years'));

interactive(children=(FloatSlider(value=0.05, description='Money Growth', max=0.15, step=0.01), FloatSlider(va…

# 💰 Cash Money and GDP

More dollars **doesn’t always mean** more goods.

When we look at GDP:
- **Nominal GDP** = Prices × Quantities
- **Real GDP** = **Quantities**, adjusted for inflation



# 🛒 Metaphor:
If you go shopping with $50 and buy:
- 10 items at $5 each → real GDP = 10

Now if prices rise to $10:
- With $100, you still only get 10 items → real GDP unchanged

But nominal GDP **doubled**.



# 🔍 Key Insight:
> **Nominal GDP growth ≠ real growth**  
> We strip out inflation to understand **what’s really changing**.